In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import spacy
from spacy.language import Language
from spacy import displacy
import time
import glob
import re
import math
import statistics
import os
import json
import calendar
import holidays
from pathlib import Path
from datetime import date
from datetime import datetime
import pandas as pd
import numpy as np
import collections
import hashlib
from dateutil.parser import parse
import shutil
import ast
from io import StringIO
import requests

In [44]:
import spacy
from spacy.language import Language
from spacy import displacy
import time

@Language.component("newsent")
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        #print(token.text, token.text in ("’s", "'s"))
        if token.text.upper() in (";", "--", "\n\n", "\n", "QUARTERLY", "STORY", "\n\n\n\n", "\n\n\n"):
            #print("Detected:", token.text)
            doc[token.i].is_sent_start = True
    return doc

#spacy.require_gpu()
nlp = spacy.load("../../Summary/NER/RelateEntity/train/model-best-local")
nlp.add_pipe('sentencizer')
nlp.add_pipe('newsent', name="newsent", last=True)

c:\users\ankan\appdata\local\programs\python\python37\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\users\ankan\appdata\local\programs\python\python37\lib\site-packages\spacy_transformers\pipeline_component.py:406: UserWarning: Automatically converting a transformer component from spacy-transformers v1.0 to v1.1+. If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spacy-transformers version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


<function __main__.set_custom_boundaries(doc)>

In [45]:
def getSentences(inputfile, nlp, text=None):
    if(not text):
        with open(inputfile, 'r', encoding="utf-8") as f:
            text = f.read()

    doc = nlp(text)
    sentences = [str(sent).strip() for sent in doc.sents]

    #print(len(sentences))
    return(sentences)

In [55]:
rplStr = ["PG*** ", "ED*** ", "SCHQ*** ", "SCBQ*** ", "SCBF*** ", "SCHF*** ", "SCG*** ", "GF*** ", "GQ*** ", "SC*** ", "NOPAD*** "]

def preProcessSent(line):
    for s in rplStr:
        line = line.replace(s, "")
    if("TBLST***" in line or "TBLET***" in line or "CS***" in line or line == "\n" or line == "\n\n" or "https://finance.yahoo.com" in line):
        return(line, False)
    line = line.replace("\n", "")
    if(line == None or line == ""):
        return(line, False)
    return(line, True)

In [2]:
import hashlib
def getHash(sent):
    hash_object = hashlib.sha1(sent.upper().encode("UTF-8"))
    hex_dig = hash_object.hexdigest()
    return(hex_dig)

In [57]:
def getRelationFromER(sym):
    entityFile = "../../Summary/entities/"+sym+"-ENTITIES.json"
    print(entityFile)
    #search = "Q\d+-[0-9][0-9][0-9][0-9]|ALL-[0-9][0-9][0-9][0-9]"
    search = "Q\d+-[0-9][0-9][0-9][0-9]"
    print(search)
    cnt = 0
    sentER = dict()
    with open(entityFile, encoding="utf-8") as f:
        entity = json.load(f)
    if(entity):
        for key in entity[sym].keys():
            if(re.search(search, key)):
                if("ALL" not in key):
                    cnt = cnt + 1
                #if(cnt == 1):
                #    continue
                if(cnt > 5):
                    break
                for ent in entity[sym][key]:
                    if("PREV" not in ent and type(entity[sym][key][ent]) == dict and "RELATION" in entity[sym][key][ent] and len(entity[sym][key][ent]["RELATION"]) > 0):
                        for r in entity[sym][key][ent]["RELATION"]:
                            newent = ent.replace("-GUIDEFULL","")
                            newent = newent.replace("-GUIDE","")
                            newent = re.sub("-"+search, "", newent)
                            sent = r.split(":::")[2]
                            sent = sent.strip()
                            hexDig = getHash(sent)
                            relation = "{METRIC:"+newent+","
                            if hexDig not in sentER:
                                sentER[hexDig] = dict()
                            else:
                                if("RELATION" in sentER[hexDig] and relation in sentER[hexDig]["RELATION"][0]):
                                    print("SAME RELATION " + relation + " ALREADY PRESENT FOR " + sent)
                                    continue
                            if("RTEXT-MONEY" in entity[sym][key][ent] and "RTEXT-RANGEM1" in entity[sym][key][ent]):
                                relation = relation + "MONEY:"+entity[sym][key][ent]["RTEXT-MONEY"]+",RANGEM1:"+entity[sym][key][ent]["RTEXT-RANGEM1"]+",RELATION:KVRANGE}"
                            elif("RTEXT-CD" in entity[sym][key][ent] and "RTEXT-RANGEM1" in entity[sym][key][ent]):
                                relation = relation + "CD:"+entity[sym][key][ent]["RTEXT-CD"]+",RANGEM1:"+entity[sym][key][ent]["RTEXT-RANGEM1"]+",RELATION:KVRANGE}"
                            elif("RTEXT-PCT" in entity[sym][key][ent] and "RTEXT-RANGEM1" in entity[sym][key][ent]):
                                relation = relation + "PCT:"+entity[sym][key][ent]["RTEXT-PCT"]+",RANGEM1:"+entity[sym][key][ent]["RTEXT-RANGEM1"]+",RELATION:KVRANGE}"
                            elif("RTEXT-MONEY" in entity[sym][key][ent]):
                                relation = relation + "MONEY:"+entity[sym][key][ent]["RTEXT-MONEY"]+",RELATION:KV}"
                            elif("RTEXT-CD" in entity[sym][key][ent]):
                                relation = relation + "CD:"+entity[sym][key][ent]["RTEXT-CD"]+",RELATION:KV}"
                            elif("RTEXT-PCT" in entity[sym][key][ent]):
                                relation = relation + "PCT:"+entity[sym][key][ent]["RTEXT-PCT"]+",RELATION:KV}"
                            #print(key, sent+"\t"+relation, hexDig)
                            sentER[hexDig]["ORIGSENT"] = sent
                            sentER[hexDig]["KEY"] = key
                            if("RELATION" not in sentER[hexDig]):
                                sentER[hexDig]["RELATION"] = list()
                            sentER[hexDig]["RELATION"].append(relation)
    #print(sentER)
    return(sentER)

In [11]:
def getFileSortTimestamp(csym, src="REFINED"):
    if(src == "POSTREFINED"):
        path = "../../Summary/PostRefined"
    else:
        path = "../../Summary/Refined"
    files = glob.glob(path+"/"+csym+"/*")   
    files.sort(reverse=True, key=os.path.getmtime)
    return(files)

In [21]:
def getOrgData(org):
    orgDataPath = "../../Summary/orgData/"+org+".txt"
    file = Path(orgDataPath)
    if file.is_file():
        #print(True)
        with open(orgDataPath) as f:
            data = json.load(f)
        #print(data)
        return data
    return None

In [22]:
def getOrgAttr(orgData, attr):
    if not orgData:
        return None
    asplit = attr.split("|")
    parent = asplit[0]
    if "SOURCE" in orgData[parent]:
        src = orgData[parent]["SOURCE"]
        if src == "YH" or (parent == "ORGPROFILE" and src == "AD"):
            p = orgData
            for i in range(0, len(asplit)):
                if asplit[i] not in p:
                    return None
                p = p[asplit[i]]
            #print(p)
            return(p)
    return None

In [23]:
def getQtrs(org, edate, fynd):
    #print(edate)
    dateObj = datetime.strptime(edate, '%Y-%m-%d')
    #print(dateObj.month)
    emonth = dateObj.month
    eyr = dateObj.year
    k = 1
    qend = fynd
    qstart = 0
    year = eyr
    #print(year)
    qtrs = dict()
    while(k!=5):
        #print(k)
        qstart = (qend + 1)%12
        qend = (qend + 3)%12
        if qstart == 0:
            qstart = 12
        if qend == 0:
            qend = 12
        if((emonth >= qstart and emonth <= qend) or (qstart > qend and (emonth >= qstart or emonth <= qend))):
            #print("CQTR:", "Q"+str(k))
            qtrs["CQTR"] = "Q"+str(k)
            qtrs["GQTR"] = "Q"+str(k) # Guidance qtr is same as current qtr
            qtrs["CYR"] = str(year)
            qtrs["GYR"] = str(year) # Guidance year same as current year
            eqtr = (k-1)%4
            if(eqtr == 0):
                eqtr = 4
            nqtr = (k+1)%4
            if(nqtr == 0):
                nqtr = 4
            pqtr = (k-2)%4
            if(pqtr == 0):
                pqtr = 4
            qtrs["EQTR"] = "Q"+str(eqtr)
            qtrs["PQTR"] = "Q"+str(pqtr)
            #print("EQTR:", "Q"+str(eqtr))
            #print("PQTR:", "Q"+str(pqtr))
            #print("NQTR:", "Q"+str(nqtr))
        #print("Q"+str(k), "START", qstart)
        #print("Q"+str(k), "END", qend)
        qtr = "Q"+str(k)
        if (qtr not in qtrs):
            qtrs[qtr] = dict()
        qtrs[qtr]["START"] = qstart
        qtrs[qtr]["SM"] = calendar.month_abbr[qstart].upper()
        qtrs[qtr]["END"] = qend
        qtrs[qtr]["EM"] = calendar.month_abbr[qend].upper()
        #print("Q"+str(k), "YEAR", year)
        k = k + 1
    
    #print(eyr)   
    if("EQTR" in qtrs and "CQTR" in qtrs and "PQTR" in qtrs):
        eqtrEnd = qtrs[qtrs["EQTR"]]["END"]
        cqtrEnd = qtrs[qtrs["CQTR"]]["END"]
        pqtrEnd = qtrs[qtrs["PQTR"]]["END"]
        #print(eqtrEnd, cqtrEnd, pqtrEnd)
        qtrs["EYR"] = str(eyr)
        qtrs["PYR"] = str(eyr)
        #print(eqtrEnd, cqtrEnd, qtrs[qtrs["CQTR"]]["START"], qtrs[qtrs["CQTR"]]["END"])
        if(eqtrEnd > cqtrEnd):
            if(qtrs[qtrs["CQTR"]]["START"] > qtrs[qtrs["CQTR"]]["END"] and emonth > fynd):
                gyr = year + 1
                qtrs["GYR"] = str(gyr)
                qtrs["EYR"] = str(gyr - 1)
            else:
                qtrs["EYR"] = str(eyr - 1)
                qtrs["PYR"] = str(eyr - 1)
        if(pqtrEnd > eqtrEnd):
            qtrs["PYR"] = str(int(qtrs["EYR"]) - 1)
    if(emonth <= fynd):
        qtrs["CFYR"] = str(year)
    else:
        qtrs["CFYR"] = str(year + 1)
    qtrs["FYR"] = qtrs["CFYR"]
    if(qtrs[qtrs["EQTR"]]["END"] <= fynd and qtrs["EQTR"] > qtrs["CQTR"]):
        qtrs["FYR"] = str(int(qtrs["FYR"]) - 1)
    qtrs["EFYR"] = qtrs["FYR"]
    qtrs["PFYR"] = qtrs["FYR"]
    if(qtrs["PQTR"] > qtrs["EQTR"]):
        qtrs["PFYR"] = str(int(qtrs["FYR"]) - 1)
    qtrs["GFYR"] = qtrs["CFYR"]
    #qtrs["EYR"] = qtrs["FYR"]
    #qtrs["CYR"] = qtrs["CFYR"]
    #qtrs["PYR"] = qtrs["FYR"]
    qtrs["CYR"] = qtrs["GYR"]
    #print(qtrs)
    return (qtrs)

In [90]:
def scanER(csym, estr, infile, sentER):
    infilePath = Path(infile)
    if not infilePath:
        print("File {} does not exisit".format(infile))
        return
    with open(infilePath, "r", encoding = "utf-8") as f:
        line = f.readline()
        #print(line)
        while line:
            #print(line)
            if("ED***" in line):
                break
            elif("PG***" in line or "NOPAD***" in line or ("***" not in line)):
                sentences = getSentences(None, nlp, line)
                for l in sentences:
                    #print(l)
                    pline, isProcess = preProcessSent(l)
                    if not isProcess:
                        continue
                    hexDig = getHash(pline)
                    if hexDig in sentER and "KEY" in sentER[hexDig] and sentER[hexDig]["KEY"] == estr:
                        print(pline+"\t"+"RELATION:"+str(sentER[hexDig]["RELATION"]))
                    else:
                        print(pline+"\t"+"RELATION:None")
            else:
                if(line == "\n"):
                    line = f.readline()
                    continue
                print(line+"\t"+"RELATION:None")
            line = f.readline()

In [91]:
csym = "APPN"
files = getFileSortTimestamp(csym)
print(files)
pfiles = getFileSortTimestamp(csym, "POSTREFINED")
print(pfiles)

#entityFile = "../../Summary/entities/"+csym+"-ENTITIES.json"
#print(entityFile)
#with open(entityFile, encoding="utf-8") as f:
#    entity = json.load(f)

sentER = getRelationFromER(csym)

orgData = getOrgData(csym)
if orgData:
    val = getOrgAttr(orgData, "ORGPROFILE|FiscalYearEnd")

cnt = 0
cdate = "2023-06-30"
for file in files:
    cnt = cnt + 1
    if(cnt > 5):
        break
    basefile = os.path.basename(file)
    filename = os.path.splitext(basefile)[0]
    #print(filename)
    edate = filename.split("_")[1]
    rtype = filename.split("_")[0]
    estr = None
    if orgData:
        if(val):
            dobj = datetime.strptime(val, "%B")
            fynd = (dobj.month)
            qtrs = getQtrs(csym, edate, fynd)
            eqtr = qtrs["EQTR"]
            eyr = qtrs["EYR"]
            estr = eqtr+"-"+eyr
    if(estr):
        nedate = time.strptime(edate, "%Y-%m-%d")
        ncdate = time.strptime(cdate, "%Y-%m-%d")
        if(nedate > ncdate):
            nfile = "../../Summary/PostRefined/"+csym+"/"+basefile
            print(edate, rtype, estr, nfile)
        else:
            nfile = file
            print(edate, rtype, estr, nfile)
        scanER(csym, estr, nfile, sentER)

['../../Summary/Refined/APPN\\APPN_2023-05-09_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2023-08-03_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2023-02-16_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2022-11-03_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2022-08-04_EP_YH.txt', '../../Summary/Refined/APPN\\APPN_2022-05-06_EP_YH.txt']
['../../Summary/PostRefined/APPN\\APPN_2023-08-03_EP_YH.txt', '../../Summary/PostRefined/APPN\\APPN_2023-05-09_EP_YH.txt', '../../Summary/PostRefined/APPN\\APPN_2023-02-16_EP_YH.txt', '../../Summary/PostRefined/APPN\\APPN_2022-11-03_EP_YH.txt', '../../Summary/PostRefined/APPN\\APPN_2022-08-04_EP_YH.txt']
../../Summary/entities/APPN-ENTITIES.json
Q\d+-[0-9][0-9][0-9][0-9]
2023-05-09 APPN Q1-2023 ../../Summary/Refined/APPN\APPN_2023-05-09_EP_YH.txt
GAAP Net Loss Per Share is $(0.51) in first quarter 2023 @@@	RELATION:['{METRIC:GAAP-EPS,MONEY:-$0.51,RELATION:KV}']
GAAP Gross Profit is $99142 T in first quarter 2023 @@@	RELATION:['{METRIC:GAAP GROSS PRO

GAAP Net Loss Per Share is $(0.58) in second quarter 2023 @@@	RELATION:['{METRIC:GAAP-EPS,MONEY:-$0.58,RELATION:KV}']
GAAP Gross Profit is $90870 T in second quarter 2023 @@@	RELATION:['{METRIC:GAAP GROSS PROFIT,MONEY:$90.9MN,RELATION:KV}']
GAAP Gross Margin is 71.15% in second quarter 2023 @@@	RELATION:['{METRIC:GAAP GROSS MARGIN,PCT:71.15%,RELATION:KV}']
.	RELATION:None
Appian Corporation .	RELATION:None
Second quarter cloud subscription revenue is $74.4 million .	RELATION:None
MCLEAN, Va., Aug. 03, 2023 (GLOBE NEWSWIRE)	RELATION:None
-- Appian (Nasdaq: APPN) today announced financial results for the second quarter ended June 30, 2023 .	RELATION:None
CS*** “Appian is leading in AI-based process automation. Our practical and private approach to AI is garnering customer and media interest, differentiating us from the competition,” said Matt Calkins, CEO & Founder.
	RELATION:None
SCHQ*** Second Quarter 2023 Financial Highlights:.
	RELATION:None
Cloud subscription revenue is $74.4 millio

PG***	RELATION:None
Story continues.	RELATION:None
SCHF*** Full Year 2022 Financial Highlights:.
	RELATION:None
Revenue: Cloud subscription revenue was $236.9 million for the full year 2022, up 32% compared to the full year 2021.	RELATION:None
Total subscriptions revenue increased 29% year-over-year to $340.2 million for the full year 2022.	RELATION:['{METRIC:SUBSCRIPTIONS REVENUE-FULLYEAR,MONEY:$340.2MN,RELATION:KV}']
Professional services revenue was $127.8 million for the full year 2022, compared to $105.5 million for the full year 2021.	RELATION:['{METRIC:PROFESSIONAL SERVICES REVENUE-FULLYEAR,MONEY:$127.8MN,RELATION:KV}']
Total revenue was $468.0 million for the full year 2022, up 27% compared to the full year 2021.	RELATION:['{METRIC:REVENUE-FULLYEAR,MONEY:$468.0MN,RELATION:KV}']
Operating loss and non-GAAP operating loss: GAAP operating loss was $(145.0) million for the full year 2022, compared to $(83.9) million for the full year 2021.	RELATION:['{METRIC:GAAP OPERATING INCOME-F

Net loss and non-GAAP net loss: GAAP net loss was $(44.0) million, compared to $(25.4) million for the third quarter of 2021.	RELATION:['{METRIC:GAAP NET INCOME,MONEY:-$44.0MN,RELATION:KV}']
GAAP net loss per share was $(0.61) for the third quarter of 2022, compared to $(0.36) for the third quarter of 2021.	RELATION:['{METRIC:GAAP-EPS,MONEY:-$0.61,RELATION:KV}']
Non-GAAP net loss was $(30.9) million, compared to $(15.9) million for the third quarter of 2021.	RELATION:['{METRIC:NON-GAAP NET INCOME,MONEY:-$30.9MN,RELATION:KV}']
Non-GAAP net loss per share was $(0.43), compared to the $(0.22) net loss per share for the third quarter of 2021.	RELATION:['{METRIC:EPS,MONEY:-$0.43,RELATION:KV}']
GAAP and non-GAAP net loss for the third quarter of 2022 included $6.1 million, or $(0.08) per share, of foreign currency exchange losses.	RELATION:None
GAAP and non-GAAP net loss for the third quarter of 2021 included $2.3 million, or $(0.03) per share, of foreign currency exchange losses.	RELATION:N

PG***	RELATION:None
A reconciliation of GAAP to non-GAAP financial measures has been provided in the tables following the financial statements in this press release.	RELATION:None
An explanation of these measures is also included below under the heading “Non-GAAP Financial Measures.”.	RELATION:None
SCBQ*** Recent Business Highlights:.
	RELATION:None
Appian named a “Customers' Choice” again in 2022 Gartner® Peer Insights™ Voice of the Customer: Enterprise Low-Code Application Platforms.	RELATION:None
Italian Postal Service improves operational efficiency by 70% with Appian.	RELATION:None
Pepper Money’s SOLANA improves business volumes by 70% with the Appian Low-Code Platform.	RELATION:None
Appian announces 2022 “Excellence in Low-Code” award winners.	RELATION:None
Appian announces #lowcode4all.	RELATION:None
Appian names Bill McCarthy and Mark Lynch to Board of Directors.	RELATION:None
Appian names new Chief People Officer and new Senior Vice President of Industry Products and Solutions

In [2]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

Downloading:   0%|          | 0.00/662 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.92G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

In [3]:
inputs = tokenizer("A step by step recipe to make bolognese pasta:", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['Toss the pasta with the sauce, then add the meat and toss again.']


In [4]:
inputs = tokenizer("Get the Named entity from this sentence: Total revenue is expected to be between $115.0 million and $117.0 million, representing a year-over-year increase of 24% to 27%.", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['Total revenue']


In [5]:
advanced_ner = """Microsoft Corporation is a company that makes computer software and video games. Bill Gates and Paul Allen founded the company in 1975 
[Company]: Microsoft, [Founded]: 1975, [Founders]: Bill Gates, Paul Allen 
 
Amazon.com, Inc., known as Amazon , is an American online business and cloud computing company. It was founded on July 5, 1994 by Jeff Bezos 
[Company]: Amazon, [Founded]: 1994, [Founders]: Jeff Bezos 
 
Apple Inc. is a multinational company that makes personal computers, mobile devices, and software. Apple was started in 1976 by Steve Jobs and Steve Wozniak."""
inputs = tokenizer(advanced_ner, return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['[Company]: Apple, [Founded]: 1976, [Founders]:']


In [6]:
report_ner = """"Revenue is $503MN up 77% year over year
[Metric]: Revenue, [Money]: $503MN, [PCT]: 77%, [CALENDAR]: year over year

Revenue is $603MN up 87% year over year
[Metric]: Revenue, [Money]: $603MN, [PCT]: 87%, [CALENDAR]: year over year

Revenue is $63MN down 12% year over year.
"""
inputs = tokenizer(report_ner, return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['"Revenue is $503MN up 77% year over year [Metric]:']
